In [2]:
import pandas as pd

# all three dataframes
dataframes = {}
dataframes["gdp_emissions"] = pd.read_csv('../final_csv/merged_df_gdp_emissions.csv')
dataframes["gdp_suicide"]  = pd.read_csv('../final_csv/merged_df_gdp_suicide.csv')
dataframes["emissions_suicide"] = pd.read_csv('../final_csv/merged_df_emissions_suicide.csv')

# for the dropdown
types = [
    {'name': 'gdp_emissions', 'text': 'GDP / Emissions'},
    {'name': 'gdp_suicide', 'text': 'GDP / Suicide'},
    {'name': 'emissions_suicide', 'text': 'Emissions / Suicide'},
]

# get the years descending of the passed dataframe
def getYears(type): 
    global dataframes
    df = dataframes[type].copy()
    years = df["year"].unique()
    years = years[::-1]

    return years

# get markers of the passed dataframe based on the passed year
def getMarkers(type, year):
    global dataframes

    markers = []
    for index, row in dataframes[type].copy().loc[dataframes[type].copy()['year'] == int(year)].iterrows():
        html = '<span class=\'country\'>'+ row['country'] +'</span><table>'
        if type == 'gdp_emissions':
            html += '<tr><td>GDP</td><td>' + str(row['GDP_Value']) + '</td></tr> <tr><td>Emissions</td><td>' + str(row['Total amount of CO2']) + '</td></tr>'
        elif type == 'gdp_suicide':
            html += '<tr><td>GDP</td><td>' + str(row['GDP_Value']) + '</td></tr> <tr><td>Suicides</td><td> ' + str(row['suicides_no']) + '</td></tr>'
        else:
            html += '<tr><td>Emissions</td><td>' + str(row['Total amount of CO2']) + '</td></tr> <tr><td>Suicides</td><td> ' + str(row['suicides_no']) + '</td></tr>'

        html += '</table>'
        markers.append({'lat':row['latitude'], 'lon':row['longitude'], 'popup': html})

    return markers

# Flask
from flask import Flask, render_template, request

app=Flask(__name__)

# URL / => return index.html with gdp and emissions data in 2016
@app.route('/')
def root():
    return render_template('index.html',markers=getMarkers('gdp_emissions', 2016), years=getYears('gdp_emissions'), types=types, activeYr=0, activeCy='')


# URL /data => return index.html with specific type and year data
@app.route('/data')
def showData():
    category = str(request.args.get('category', default = 'gdp_emissions', type = str))
    year = str(request.args.get('year', default = 2016, type = int))
    return render_template('index.html',markers=getMarkers(category, year), years=getYears(category),types=types, activeYr=int(year), activeCy=str(category))

if __name__ == '__main__':
    app.run(host="localhost", port=8080, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8080
Press CTRL+C to quit
127.0.0.1 - - [28/May/2023 18:11:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2023 18:11:28] "GET /static/universal.js HTTP/1.1" 304 -
127.0.0.1 - - [28/May/2023 18:11:28] "GET /static/universal.css HTTP/1.1" 304 -
